# Microsoft Movie Studios Data Analysis

In [1]:
# Here is the template I am following for the final Jupyter Notebook: 
# Link: https://github.com/learn-co-curriculum/dsc-project-template/blob/example-aab/animal_shelter_needs_analysis.ipynb

## Overview

In [2]:
#overview of project goes here

## Business Problem

In [3]:
#business problem goes here

# Importing Libraries & Specifying File Paths

In [4]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
%matplotlib inline

In [5]:
# Specifying all file paths in one centralized location.
tn_budgets_file_path = '~/Documents/Flatiron/phase_1/Movie_Studio_DATA/tn.movie_budgets.csv.gz'
imdb_file_path = './../Movie_Studio_DATA/im.db'
movie_budget_file_path = '~/Documents/Flatiron/phase_1/Movie_Studio_DATA/tn.movie_budgets.csv.gz'
microsoft_movie_studio_db_file_path = './../Movie_Studio_DATA/im.db'
msms_db_conn = sqlite3.connect(microsoft_movie_studio_db_file_path)


## Data Understanding

Runtime Data Exploration

In [6]:
#Loading in datasets for runtime analysis
#Using pandas to create a dataframe from the tn.movie_budgets csv file
tn_budgets_df = pd.read_csv(tn_budgets_file_path)

In [7]:
# ensuring that the file was loaded correctly
tn_budgets_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [8]:
# creating a sqlite3 connection to be able to read in our SQL database 
conn = sqlite3.connect(imdb_file_path) 

# using pandas to read in our SQL database im.db
# SELECT * because we need to see what tables are in this database
imdb_db = pd.read_sql('''
                               SELECT
                               *
                               FROM sqlite_master
                               ''', conn)

In [9]:
# take a look at our dataframe to make sure it was read correctly
imdb_db

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [10]:
# Create a dataframe with the columns needed from the movie_basics table
imdb_table = pd.read_sql('''
                            SELECT
                               movie_basics.movie_id, movie_basics.primary_title, movie_basics.runtime_minutes
                               FROM movie_basics
                               INNER JOIN movie_akas ON movie_basics.movie_id = movie_akas.movie_id
                                WHERE is_original_title != 0.0
                               ''', conn)

In [11]:
# take a look at our dataframe to make sure it was read correctly
imdb_table.head()

,movie_id,primary_title,runtime_minutes
0,tt0369610,Jurassic World,124.0
1,tt0401729,John Carter,132.0
2,tt10010134,Versailles Rediscovered - The Sun King's Vanis...,52.0
3,tt10027708,Miguelito,94.0
4,tt10050722,Thing I Don't Get,71.0


Writer/Director Data Exploration

In [12]:
# Writer/Director Data Understanding Starts:
movie_basics_table = pd.read_sql('''
                               SELECT 
                               movie_basics.movie_id,movie_basics.primary_title
                               FROM movie_basics
                               INNER JOIN movie_akas ON movie_basics.movie_id = movie_akas.movie_id
                               WHERE is_original_title != 0.0
                               ''', conn)

In [13]:
movie_basics_table.head()

,movie_id,primary_title
0,tt0369610,Jurassic World
1,tt0401729,John Carter
2,tt10010134,Versailles Rediscovered - The Sun King's Vanis...
3,tt10027708,Miguelito
4,tt10050722,Thing I Don't Get


In [14]:
persons_table = pd.read_sql('''
                               SELECT
                               person_id,primary_name,death_year
                               FROM persons
                               ''', conn)

In [15]:
persons_table.head()

,person_id,primary_name,death_year
0,nm0061671,Mary Ellen Bauder,NaN
1,nm0061865,Joseph Bauer,NaN
2,nm0062070,Bruce Baum,NaN
3,nm0062195,Axel Baumann,NaN
4,nm0062798,Pete Baxter,NaN


In [16]:
writers_table = pd.read_sql('''
                               SELECT
                               *
                               FROM writers
                               ''', conn)

In [17]:
writers_table.head()

,movie_id,person_id
0,tt0285252,nm0899854
1,tt0438973,nm0175726
2,tt0438973,nm1802864
3,tt0462036,nm1940585
4,tt0835418,nm0310087


In [ ]:
writers_df = pd.merge(writers_table, persons_table,left_on='person_id',right_on='person_id')

In [ ]:
writers_df.head()

In [ ]:
writers_df = pd.merge(writers_df,movie_basics_table,left_on='movie_id',right_on='movie_id')

In [ ]:
writers_df.head()

In [ ]:
writers_df = pd.merge(tn_budgets_df,writers_df, how='inner',left_on='movie',right_on='primary_title')

In [ ]:
directors_table = pd.read_sql('''
                               SELECT
                               *
                               FROM directors
                               ''', conn)

In [ ]:
directors_table.head()

In [ ]:
directors_df = pd.merge(directors_table, persons_table,left_on='person_id',right_on='person_id')

In [ ]:
directors_df = pd.merge(directors_df,movie_basics_table,left_on='movie_id',right_on='movie_id')

In [ ]:
directors_df.head()

In [ ]:
directors_df = pd.merge(tn_budgets_df,directors_df, how='inner',left_on='movie',right_on='primary_title')

Genre Data Exploration

In [ ]:
#reading in movie budget csv for outcome data
movie_budget_df = pd.read_csv(movie_budget_file_path)
#check to see if read properly
movie_budget_df.head()

In [ ]:
#execute SQL query to obtain a dataframe with movie names and avg ratings, filtered for original titles
genre_query = """
SELECT movie_basics.primary_title, movie_basics.start_year, movie_basics.genres
FROM movie_basics
INNER JOIN movie_akas ON movie_basics.movie_id = movie_akas.movie_id
WHERE is_original_title != 0.0
"""

filtered_genre_info_df = pd.read_sql(genre_query, msms_db_conn)

#check query results
filtered_genre_info_df

In [ ]:
#attempt to merge genre data with budget data for comparison of dependent/independent variables
genre_budget_df = pd.merge(filtered_genre_info_df, 
                                  movie_budget_df, 
                                  how='inner', 
                                  left_on='primary_title', 
                                  right_on='movie' )
genre_budget_df

In [ ]:
#Identified a number of duplicate entries
genre_budget_df["primary_title"].value_counts()

# Data cleaning and preparation

Runtime Data Cleaning:

In [ ]:
# Cleaning the production_budget column by removing all of the commas and the dollar sign then converting the 
# value to a float
tn_budgets_df['production_budget_float'] = tn_budgets_df['production_budget'].str.replace(",","")
tn_budgets_df['production_budget_float'] = tn_budgets_df['production_budget_float'].str[1:]
tn_budgets_df['production_budget_float'] = tn_budgets_df['production_budget_float'].astype(float)

In [ ]:
# Cleaning the domestic_gross column by removing all of the commas and the dollar sign then converting the 
# value to a float
tn_budgets_df['domestic_gross_float'] = tn_budgets_df['domestic_gross'].str.replace(",","")
tn_budgets_df['domestic_gross_float'] = tn_budgets_df['domestic_gross_float'].str[1:]
tn_budgets_df['domestic_gross_float'] = tn_budgets_df['domestic_gross_float'].astype(float)

In [ ]:
# Cleaning the worldwide_gross column by removing all of the commas and the dollar sign then converting the 
# value to a float
tn_budgets_df['worldwide_gross_float'] = tn_budgets_df['worldwide_gross'].str.replace(",","")
tn_budgets_df['worldwide_gross_float'] = tn_budgets_df['worldwide_gross_float'].str[1:]
tn_budgets_df['worldwide_gross_float'] = tn_budgets_df['worldwide_gross_float'].astype(float)

In [ ]:
# Deleting all the rows that have a value of zero in the columns 'production_budget_float','worldwide_gross_float' 
# or 'domestic_gross_float'
tn_budgets_df = tn_budgets_df[tn_budgets_df['production_budget_float'] != 0]
tn_budgets_df = tn_budgets_df[tn_budgets_df['worldwide_gross_float'] != 0]
tn_budgets_df = tn_budgets_df[tn_budgets_df['domestic_gross_float'] != 0]

#sanity check
#tn_budgets_df.loc[tn_budgets_df['domestic_gross_float'] == 0]
#tn_budgets_df.loc[tn_budgets_df['worldwide_gross_float'] == 0]
#tn_budgets_df.loc[tn_budgets_df['production_budget_float'] == 0]

In [ ]:
# creating a new column named 'net_income' that is equal to the worldwide gross minus the production budget
tn_budgets_df['net_income'] = tn_budgets_df['worldwide_gross_float'] - tn_budgets_df['production_budget_float']

In [ ]:
# Taking a look at tn_budgets_df
tn_budgets_df.head()

In [ ]:
# Joining the data frams 'tn_budgets_df and 'imdb_table' on the movie title
title_and_revenue_df = pd.merge(tn_budgets_df, imdb_table,left_on='movie',right_on='primary_title')
title_and_revenue_df.head()

In [ ]:
# Here I am dropping all the rows in our dataframe where 'net_income' is NAN
title_and_revenue_df = title_and_revenue_df.dropna(subset=['net_income'])

# Here I am dropping all the duplicates
title_and_revenue_df = title_and_revenue_df.drop_duplicates(subset='movie')

#sanity checks
#title_and_revenue_df['movie'].value_counts()
#title_and_revenue_df['movie'].isnull().sum()

Director/Writer Data Cleaning:

In [ ]:
#Director/Writer Data Cleaning and Preparation Start:
# Here I am dropping all the rows in our dataframe where 'movie' is NAN
writers_df = writers_df.dropna(subset=['movie'])

# Here I am dropping all the duplicates
writers_df = writers_df.drop_duplicates(subset='movie')

#sanity checks
#writers_df['movie'].value_counts()
#writers_df['movie'].isnull().sum()


In [ ]:
# Need to ensure that the writers are still alive
# Dropping all rows where death_year != Nan
writers_df = writers_df.loc[writers_df['death_year'].isna()]

In [ ]:
print(len(writers_df))
print(writers_df['death_year'].isna().sum())

In [ ]:
new_writers = writers_df.groupby(by=['primary_name'],as_index=False).mean()

In [ ]:
new_writers = new_writers.sort_values(by=['net_income'],ascending=False)
new_writers.head()
new_writers = new_writers[:10]

In [ ]:
# Here I am dropping all the rows in our dataframe where 'movie' is NAN
directors_df = directors_df.dropna(subset=['movie'])

# Here I am dropping all the duplicates
directors_df = directors_df.drop_duplicates(subset='movie')

#sanity checks
#directors_df['movie'].value_counts()
#directors_df['movie'].isnull().sum()

In [ ]:
# Need to ensure that the directors are still alive
# Dropping all rows where death_year != Nan
directors_df = directors_df.loc[directors_df['death_year'].isna()]

In [ ]:
print(len(directors_df))
print(directors_df['death_year'].isna().sum())

In [ ]:
new_directors = directors_df.groupby(by=['primary_name'],as_index=False).mean()

In [ ]:
new_directors.head()

In [ ]:
new_directors = new_directors.sort_values(by=['net_income'],ascending=False)
new_directors = new_directors[:10]
new_directors.head()

Genre Data Cleaning:

In [ ]:
#Removing duplicate entries
genre_budget_df.drop_duplicates(subset="primary_title", inplace=True)
genre_budget_df

In [ ]:
#removing special characters from columns for conversion to numeric data type
genre_budget_df['production_budget'] = genre_budget_df['production_budget'].replace(",","",regex=True)
genre_budget_df['production_budget'] = genre_budget_df['production_budget'].str.replace("$","",regex=True)

genre_budget_df['domestic_gross'] = genre_budget_df['domestic_gross'].replace(",","",regex=True)
genre_budget_df['domestic_gross'] = genre_budget_df['domestic_gross'].str.replace("$","",regex=True)

genre_budget_df['worldwide_gross'] = genre_budget_df['worldwide_gross'].replace(",","",regex=True)
genre_budget_df['worldwide_gross'] = genre_budget_df['worldwide_gross'].str.replace("$","",regex=True)

#check replacement
genre_budget_df.head()

In [ ]:
#convert money columns to floating point values
genre_budget_df['production_budget'] = genre_budget_df['production_budget'].astype(float)
genre_budget_df['domestic_gross'] = genre_budget_df['domestic_gross'].astype(float)
genre_budget_df['worldwide_gross'] = genre_budget_df['worldwide_gross'].astype(float)

In [ ]:
#remove rows with non-null-type null values
genre_budget_df = genre_budget_df[genre_budget_df['production_budget'] != 0] 
genre_budget_df = genre_budget_df[genre_budget_df['domestic_gross'] != 0]
genre_budget_df = genre_budget_df[genre_budget_df['worldwide_gross'] != 0]
#reevaluate the data
genre_budget_df

In [ ]:
#create singular genres column, split genre strings along comma delimiter, 
#and return a column storing a list of genres stored as string values
genre_budget_df['singular_genres'] = genre_budget_df["genres"].str.split(",")

In [ ]:
#create new dataframe containing exploded singular genres as new rows
little_boy_df = genre_budget_df.explode("singular_genres")

In [ ]:
#add columns for foreign gross revenues, as well as net domestic, total, and foreign profits.
little_boy_df["foreign_market_gross"] = little_boy_df["worldwide_gross"] - little_boy_df["domestic_gross"]
little_boy_df["domestic_net"] = little_boy_df["domestic_gross"] - little_boy_df["production_budget"]
little_boy_df["worldwide_net"] = little_boy_df["worldwide_gross"] - little_boy_df["production_budget"]
little_boy_df["foreign_market_net"] = little_boy_df["foreign_market_gross"] - little_boy_df["production_budget"]

# Analysis

Runtime Analysis

In [ ]:
# Here I am taking a look at the minimum & maximum values in the 'runtime_minutes' column
max_runtime = title_and_revenue_df['runtime_minutes'].max()
min_runtime = title_and_revenue_df['runtime_minutes'].min()

# Here I am creating a new series named run_time and sorting it, and taking a look at the min and max values
run_time = title_and_revenue_df['runtime_minutes']
run_time = run_time.sort_values(ascending=True)
print(run_time.head(10))
print(min_runtime,max_runtime)


In [ ]:
fig, ax = plt.subplots(figsize=(7,5))
ax.scatter(x = title_and_revenue_df['runtime_minutes'],y = title_and_revenue_df['net_income'])
ax.set_xlabel('Runtime in Minutes')
ax.set_ylabel('Net Income for Movie')
ax.set_title('Runtime vs Net Income')
#plt.scatter(title_and_revenue_df['runtime_minutes'],title_and_revenue_df['gross_revenue']);

In [ ]:
# Here I am taking a look at the rows where there wasa net loss
title_and_revenue_df.loc[title_and_revenue_df['net_income'] < 0]

In [ ]:
title_and_revenue_df.describe()

In [ ]:
#binning by standard deviation, which you can see above - 18 minutes

def length(val):
    if val <= 18:
        return ">= 18 minutes"
    elif (val > 18) and (val <= 36):
        return "18 to 36 minutes"
    elif(val > 36) and (val <= 54):
        return "36 minutes to 54 minutes"
    elif (val > 54) and (val <= 72):
        return "54 minutes to 72 minutes"
    elif (val > 72) and (val <= 90):
        return "72 minutes to 90 minutes"
    elif (val > 90) and (val <= 108):
        return "90 minutes to 108 minutes"
    elif (val > 108) and (val <= 126):
        return "108 minutes to 126 minutes"
    elif (val > 126) and (val <= 144):
        return "126 minutes to 144 minutes"
    elif (val > 144) and (val <= 162):
        return "144 minutes to 162 minutes"
    elif (val > 162) and (val <= 180):
        return "162 minutes to 180 minutes"

In [ ]:
# mapping runtime minutes to bins
title_and_revenue_df['runtime_minutes_binned'] = title_and_revenue_df['runtime_minutes'].map(length)

In [ ]:
new_df = title_and_revenue_df.groupby(by=['runtime_minutes_binned'],as_index=False).mean()
new_df = new_df.sort_values(by='net_income',ascending=False)

In [ ]:
#plot these bins
fig,ax = plt.subplots(figsize = (10,10))
ax.bar(new_df['runtime_minutes_binned'],new_df['net_income'],color='blue')
ax.set_xlabel('Runtime (minutes)')
ax.set_ylabel('Total Net Sales (100 Millions USD)')
ax.set_title('Runtime (minutes) vs.Total Net Sales (100 Millions USD)')
plt.xticks(rotation=90);

Director/Writer Analysis

In [ ]:
#Director/Writer Analysis Start:
fig,ax = plt.subplots(figsize=(17,5))
ax.bar(new_writers['primary_name'],new_writers['net_income'],color='blue')
ax.set_xlabel('Writer')
ax.set_ylabel('Average Movie Net Income (100 Millions USD)')
ax.set_title('Top 10 Writers by Average Movie Net Income');

In [ ]:
fig,ax = plt.subplots(figsize=(17,5))
ax.bar(new_directors['primary_name'],new_directors['net_income'],color='blue')
ax.set_xlabel('Director')
ax.set_ylabel('Average Movie Net Income (100 Millions USD)')
ax.set_title('top 10 Directors by Average Movie Net Income');

Genre Data Analysis

In [ ]:
#aggregate new dataframe by singular genres
genre_average_df = little_boy_df.groupby(by="singular_genres", as_index=False).mean()

In [ ]:
#sort and save a series of dataframes so that graphs are produced with bars ordered by y-values
worldwide_gross = genre_average_df.sort_values(by="worldwide_gross", ascending=False)
domestic_gross = genre_average_df.sort_values(by="domestic_gross", ascending=False)
foreign_gross = genre_average_df.sort_values(by="foreign_market_gross", ascending=False)
worldwide_net = genre_average_df.sort_values(by="worldwide_net", ascending=False)
domestic_net = genre_average_df.sort_values(by="domestic_net", ascending=False)
foreign_net = genre_average_df.sort_values(by="foreign_market_net", ascending=False)

In [ ]:
#graph of total gross sales vs. genre
fig, ax = plt.subplots(figsize=(10,10))

x = worldwide_gross["singular_genres"]
y = worldwide_gross["worldwide_gross"]

ax.bar(x,y)
ax.set_title("Movie Genres by Average Total Gross Profits")
ax.set_xlabel("Genres")
ax.set_ylabel("Total Gross Sales (100 Millions USD)")
plt.xticks(rotation=90);

In [ ]:
#graph of total net sales vs. genre
fig, ax = plt.subplots(figsize=(10,10))

x = worldwide_net["singular_genres"]
y = worldwide_net["worldwide_net"]

ax.bar(x,y,color='green')
ax.set_title("Movie Genres by Average Total Net Profits")
ax.set_xlabel("Genres")
ax.set_ylabel("Total Net Sales (100 Millions USD)")
plt.xticks(rotation=90);

In [ ]:
#graph of domestic gross sales vs. genre
fig, ax = plt.subplots(figsize=(10,10))

x = domestic_gross["singular_genres"]
y = domestic_gross["domestic_gross"]

ax.bar(x,y)
ax.set_title("Movie Genres by Average Domestic Gross Profits")
ax.set_xlabel("Genres")
ax.set_ylabel("Domestic Gross Sales (100 Millions USD)")
plt.xticks(rotation=90);

In [ ]:
#graph of domestic net sales vs. genre
fig, ax = plt.subplots(figsize=(10,10))

x = domestic_net["singular_genres"]
y = domestic_net["domestic_net"]

ax.bar(x,y,color='green')
ax.set_title("Movie Genres by Average Domestic Net Profits")
ax.set_xlabel("Genres")
ax.set_ylabel("Domestic Net Sales (100 Millions USD)")
plt.xticks(rotation=90);

In [ ]:
#graph of foreign gross sales vs. genre
fig, ax = plt.subplots(figsize=(10,10))

x = foreign_gross["singular_genres"]
y = foreign_gross["foreign_market_gross"]

ax.bar(x,y)
ax.set_title("Movie Genres by Average Foreign Gross Profits")
ax.set_xlabel("Genres")
ax.set_ylabel("Foreign Gross Sales (100 Millions USD)")
plt.xticks(rotation=90);

In [ ]:
#graph of foreign net sales vs. genre
fig, ax = plt.subplots(figsize=(10,10))

x = foreign_net["singular_genres"]
y = foreign_net["foreign_market_net"]

ax.bar(x,y,color='green')
ax.set_title("Movie Genres by Average Foreign Net Profits")
ax.set_xlabel("Genres")
ax.set_ylabel("Foreign Net Sales (100 Millions USD)")
plt.xticks(rotation=90);

In [ ]:
#Market performance comparison chart for gross revenues

#variables/data inputs
x = genre_average_df["singular_genres"]
bar1 = genre_average_df["worldwide_gross"]
bar2 = genre_average_df["foreign_market_gross"]
bar3 = genre_average_df["domestic_gross"]

#formatting
barwidth = 0.25
r1 = np.arange(len(bar1))
r2 = [z + barwidth for z in r1]
r3 = [z + barwidth for z in r2]

#creating the chart
fig = plt.subplots(figsize=(15,15))
plt.bar(r1, bar1, color='black', width=barwidth, edgecolor='white', label='Total')
plt.bar(r2, bar2, color='red', width=barwidth, edgecolor='white', label='Foreign')
plt.bar(r3, bar3, color='blue', width=barwidth, edgecolor='white', label='Domestic')
#labelling
plt.title("Average Gross Sales by Genre in Foreign and Domestic Markets")
plt.xlabel("Genres")
plt.ylabel("Average Gross Sales by Market (100 Millions USD)")
plt.xticks([a + barwidth for a in range(len(bar1))], x, rotation=90)
plt.legend();

In [ ]:
#Market performance comparison chart for net profits

#variables/data inputs
x = genre_average_df["singular_genres"]
bar1 = genre_average_df["worldwide_net"]
bar2 = genre_average_df["foreign_market_net"]
bar3 = genre_average_df["domestic_net"]

#formatting
barwidth = 0.25
r1 = np.arange(len(bar1))
r2 = [z + barwidth for z in r1]
r3 = [z + barwidth for z in r2]

#creating the chart
fig = plt.subplots(figsize=(15,15))
plt.bar(r1, bar1, color='brown', width=barwidth, edgecolor='white', label='Total')
plt.bar(r2, bar2, color='green', width=barwidth, edgecolor='white', label='Foreign')
plt.bar(r3, bar3, color='orange', width=barwidth, edgecolor='white', label='Domestic')
#labelling
plt.title("Average Net Profit by Genre in Foreign and Domestic Markets")
plt.xlabel("Genres")
plt.ylabel("Average Net Profit by Market (100 Millions USD)")
plt.xticks([a + barwidth for a in range(len(bar1))], x, rotation=90)
plt.legend();

# Conclusions

# Next Steps